<a href="https://www.kaggle.com/code/wassimchouchen/best-model-for-ner-d-2-1?scriptVersionId=97787996" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [4]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

# Preprocessing

In [5]:
text=data["transcription_text"]

In [6]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [7]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [9]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:02<00:00,  9.14it/s]


In [10]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 2220.97it/s]


In [11]:
text[1]

'I E C Whaley solemnly swear I carefully read Amnesty Proclamation issued Andrew Johnson President of United State of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made except E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben Maj 47th PM United States of America E C Whaley of city county Charleston Sate of South Ca solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of SlavesSO HELP ME GOD E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben May 47 PM The fair complexion light hair gray eye 5 foot 10 inch high 39 year profession planter Copy Levi Steuben Maj 47th Pro Marshal City of Charleston'

In [12]:
text[10]

'AMNESTY OATH Mississippi City Miss Sept 29 1865 I JW Baxter solemnly swear presence of Almighty God I henceforth faithfully support protect defend Constitution of United States union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference emancipation of slave SO HELP ME GOD JW Baxter Sworn subscribed Mississippi City 29th day of September 1865 J H Mathews Capt 66th US Col Infty Provost Marshal Sub port Miss DESCRIPTION Age 24 year Height 5ft 8 inch Eyes Blue Color of Hair light Characteristics HEADQUARTERS Mississippi City Miss Sept 29th 1865 I hereby certify Amnesty Oath prescribed Proclamation of President of United States duly administered JW Baxter of Harrison Co order 29th day of Sept 29 1865 Frank E Miller Maj 66th USCI Commanding'

# spacy

In [13]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

In [14]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [15]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])
        
NER(text)
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])

df1=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()

,entities,start_char,end_char,label,explain,frequency
0,Anne H Elliott,11,25,PERSON,"People, including fictional",
1,Amnesty,54,61,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,82,96,PERSON,"People, including fictional",
3,United States of America,110,134,GPE,"Countries, cities, states",
4,May 29th 1865,135,148,DATE,Absolute or relative dates or periods,


In [16]:
Person=[]
ORG=[]
DATE=[]
GPE=[]

for i in range(df1.shape[0]):
    if df1.label.loc[i]=="PERSON":
        Person.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="ORG" :
        ORG.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="DATE" :
        DATE.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="GPE" :
        GPE.append(df1.entities.loc[i])
GPE=np.array(GPE)
ORG=np.array(ORG)
Person=np.array(Person)
DATE=np.array(DATE)

In [17]:
DATE

array(['May 29th 1865', '27th day', '1865', '27th day', '1865', '63 year',
       'May 29th 1865', '28 day', '28 day', 'May 47 PM', '39 year',
       'May 29th 1865', '31 day', '1865', 'No1984', '31 day', '1865',
       '31st day', 'August 24 1865', 'May 29th 1865',
       '3 day of August 1865', 'May 4', '3 day', '1865', 'May 29th 1865',
       '16th day', '1474', '16th day', 'May 29th 1865',
       '16 day of October 1865', '16 day of October 1865', 'July 22',
       'July 22', '1867', 'May 29th 1865', '6 day', '6 day',
       '29th day of September 1865', 'Age 24 year', 'Sept 29th 1865',
       '29th day', 'May 29th 1865', '7th day of September',
       '7th day of September', '20th day of August 1865',
       '20th day of August 1865', 'August 29th 1865',
       '22nd day of August 1865', 'Miss 31st day', '1865 1865', '31 day',
       '1865'], dtype='<U26')

In [18]:
len(Person)

41

In [19]:
Person

array(['Anne H Elliott', 'Andrew Johnson', 'Hazel', 'Whaley',
       'Andrew Johnson', 'Copy Levi Steuben', 'Hu Kennedy',
       'Amnesty Oath Administered At', 'Hiserodt', 'William Whaley Sworn',
       'William Whaley Sworn', 'Brown', 'Charleston Sc',
       'JF Steinmeyer Sworn', 'Charleston Sc', 'Gry', 'Joseph Edings',
       'Thomas Andrew Oath of', 'Jacob Schroder Sworn', 'Walhalla',
       'Walhalla', 'Bray Lieut', 'Laborer', 'Characteristics',
       'Amnesty Oath', 'Woodville Wilkinson', 'Frank Conner',
       'Triplicate AMNESTY', 'Hiram Smith', 'Hiram Smith',
       'Pass Christian', 'Captn', 'Amnesty Oath', 'Hiram Smith',
       'Captn Commanding', 'Robert A',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Gibbs Henderson', 'City Miss', 'Amnesty Oath', 'Frank E Miller'],
      dtype='<U58')

# Hugging face  : Flair

**ner-english** 

In [20]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.22 MiB/s, done.


In [21]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 07:51:32,742 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-08 07:51:56,084 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [22]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-08 07:52:19,194 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-08 07:52:22,122 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [23]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 07:54:01,768 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-08 07:54:20,027 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [24]:
from flair.data import Sentence
from flair.models import SequenceTagger
T2=[]
D2={}
for i in range(20):
        sentencess =Sentence(text[i])
        tagger.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T2.append(entity)
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                         D2[entity.text]= entity.get_label("ner").value

In [25]:
T3=[]
D3={}

for i in range(20):
        sentencess =Sentence(text[i])
        tagger1.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T3.append(entity)
                     if (entity.get_label("ner").score > 0.90)  :
                         D3[entity.text]= entity.get_label("ner").value

In [26]:
T4=[]
D4={}

for i in range(20):
        sentencess =Sentence(text[i])
        tagger2.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T4.append(entity)
                     if (entity.get_label("ner").score > 0.90)  :
                         D4[entity.text]= entity.get_label("ner").value

In [27]:
len(D2)

104

In [28]:
len(D3)

22

In [29]:
len(D4)

108

In [30]:
ent=[]
label=[]
for i in D2 :
    ent.append(i)
    label.append(D2[i])

In [31]:
ent1=[]
label1=[]
for i in D3 :
    ent1.append(i)
    label1.append(D3[i])

In [32]:
ent2=[]
label2=[]
for i in D4 :
    ent2.append(i)
    label2.append(D4[i])

In [33]:
final=pd.DataFrame({"entities":ent, "label":label})

In [34]:
final1=pd.DataFrame({"entities":ent1, "label":label1})

In [35]:
final2=pd.DataFrame({"entities":ent2, "label":label2})

In [36]:
P=[]
OR=[]
GP=[]
Date=[]
LOC=[]
for i in range(len(D2)):
    if final.label.loc[i]=="PERSON":
        P.append(final.entities.loc[i])
    elif final.label.loc[i]=="ORG":
        OR.append(final.entities.loc[i])
    elif final.label.loc[i]=="GPE":
        GP.append(final.entities.loc[i])
    elif final.label.loc[i]=="DATE":
        Date.append(final.entities.loc[i])
    elif final.label.loc[i]=="LOC":
        LOC.append(final.entities.loc[i])
        

OR=np.array(OR)
P=np.array(P)
LOC=np.array(LOC)
Date=np.array(Date)
GP=np.array(GP)



In [37]:
P1=[]
OR1=[]
LOC1=[]
for i in range(len(D3)):
    if final1.label.loc[i]=="PER":
        P1.append(final1.entities.loc[i])
    elif (final1.label.loc[i]=="ORG") | (final1.label.loc[i]=="MISC"):
        OR1.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="LOC" :
        LOC1.append(final1.entities.loc[i])
        

OR1=np.array(OR1)
P1=np.array(P1)
LOC1=np.array(LOC1)


In [38]:
P2=[]
OR2=[]
LOC2=[]
for i in range(len(D4)):
    if final2.label.loc[i]=="PER":
        P2.append(final2.entities.loc[i])
    elif (final2.label.loc[i]=="ORG") | (final2.label.loc[i]=="MISC"):
        OR2.append(final2.entities.loc[i])
    elif final2.label.loc[i]=="LOC" :
        LOC2.append(final2.entities.loc[i])
        

OR2=np.array(OR2)
P2=np.array(P2)
LOC2=np.array(LOC2)


In [39]:
len(Person)

41

In [40]:
len(P)

40

In [41]:
len(P1)

15

In [42]:
len(P2)

45

In [43]:
np.unique(Person)

array(['Amnesty Oath', 'Amnesty Oath Administered At', 'Andrew Johnson',
       'Anne H Elliott', 'Bray Lieut', 'Brown', 'Captn',
       'Captn Commanding', 'Characteristics', 'Charleston Sc',
       'City Miss', 'Copy Levi Steuben', 'Frank Conner', 'Frank E Miller',
       'Gibbs Henderson', 'Gry', 'Hazel', 'Hiram Smith', 'Hiserodt',
       'Hu Kennedy', 'JF Steinmeyer Sworn', 'Jacob Schroder Sworn',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Joseph Edings', 'Laborer', 'Pass Christian', 'Robert A',
       'Thomas Andrew Oath of', 'Triplicate AMNESTY', 'Walhalla',
       'Whaley', 'William Whaley Sworn', 'Woodville Wilkinson'],
      dtype='<U58')

In [44]:
np.unique(P)

array(['ANDREW JOHNSON', 'Aiken Burnwell', 'Andrew Johnson',
       'Anne H Elliott', 'Bergevin', 'Caroline Belitzer', 'City', 'E',
       'E C Whaley', 'Frank Conner', 'Frank E Miller', 'Frank Geise',
       'Geo D Carpenter', 'Hiram Smith', 'Holwell', 'Hu Kennedy',
       'I E C Whaley', 'I J F Steinmeyer', 'J E Thuring', 'J H Mathews',
       'JB Hiserodt', 'JF Steinmeyer', 'JH Mathews', 'JW Baxter',
       'Jacob Schroder', 'Jacob Schroder Sworn', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Levi Steuben', 'Levi Stuber',
       'Levi Stuber Major', 'P H Joor', 'Robert A Wilson',
       'S Wilde Durson', 'Vicksburg', 'Whaley', 'William Whaley Sworn',
       'caroline Belitzer', 'mark Sworn'], dtype='<U20')

In [45]:
np.unique(P1)

array(['Andrew Johnson', 'Anne H', 'Anne H Elliott', 'Caroline Belitzer',
       'E Montgomery', 'Frank Conner', 'Hiram Smith', 'J H Mathews',
       'Jacob Schroder', 'James E Thuring', 'Joseph Edings', 'Marshal',
       'Robert A Wilson', 'Sworn', 'Wm Whaley'], dtype='<U17')

In [46]:
np.unique(P2)

array(['ANDREW JOHNSON', 'Almighty', 'Almighty God', 'Andrew Johnson',
       'Anne H Elliott', 'Caroline Belitzer', 'E C Whaley', 'EB Towne',
       'Frank Conner', 'Frank E Miller', 'Frank Geise', 'GOD',
       'Geo D Carpenter', 'God', 'H Elliott', 'H F Holwell',
       'Hiram Smith', 'Hu Kennedy', 'I E C Whaley', 'I J F Steinmeyer',
       'J E Thuring', 'J H Mathews', 'JB Hiserodt', 'JF Steinmeyer',
       'JH Mathews', 'JW Baxter', 'Jacob Schroder', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Joseph Edings Sworn',
       'Levi Steuben', 'Levi Stuber', 'Mrs E Montgomery', 'P H Joor',
       'P P Bergevin', 'R A Bray', 'Robert A Wilson', 'S Wilde Durson',
       'S Wilde Durson Magt', 'S Wilde Durson Magtr', 'Sgd Montgomery',
       'William Whaley', 'Wm Whaley', 'caroline Belitzer'], dtype='<U20')

****

In [47]:
np.unique(ORG)

array(['AMNESTY', 'ANDREW', 'Amnesty', 'Amnesty Proclamation',
       'Belitzer Sworn', 'Charleston State of So Carolina',
       'Color of Eyes Blue Color of Hair Dark Characteristics',
       'Emancipation of Slaves', 'Eyes Blue Color of Hair',
       'Eyes Blue Color of Hair Light Characteristics', 'H F Holwell of',
       'H Mathews Capt', 'JF Steinmeyer Sworn', 'Joseph Edings Sworn',
       'Lawyer', 'Levi Stuber',
       'Magistrate S Wilde Durson AHE AHE AHE H Elliott',
       'OATH I P P Bergevin',
       'P H Joor Justice of Peace Wilkinson County State of',
       'Rebellion reference Emancipation of Slaves',
       'Regt U S C Infy Provost Marshal', 'Sgd Montgomery Sworn', 'Sworn',
       'UNITED STATES FORCES No 1282 PROVOST MARSHALS OFFICE',
       'US Col Infty Provost Marshal', 'USCI Commanding',
       'United State of America', 'United States Union',
       'United States Union of State', 'Whaley Sworn', 'Wilson'],
      dtype='<U53')

In [48]:
np.unique(OR1)

array(['James D Kendall Amnesty Oath Administered At Oacen Springs Miss 18th',
       'WA Fleitas Amnesty Oath Administered At Pass Christian Mississippi By JB Hiserodt'],
      dtype='<U81')

In [49]:
np.unique(OR2)

array(['128th USCT', '47th Pa', 'AMNESTY OATH', 'Amnesty', 'Amnesty Oath',
       'Amnesty Proclamation', 'Constitution', 'Emancipation of Slaves',
       'Emancipation of SlavesSO', 'Emancipation of SlavesSo',
       'MARSHALS OFFICE', 'NYVV', 'Oath',
       'Thomas Andrew Oath of Loyalty Recd Bureau',
       'Thos Gibbs Henderson Co', 'U S C', 'UNITED STATES FORCES', 'US',
       'US Cold Inf', 'US Cold Infty', 'USCI', 'USCT', 'Union', 'Vols',
       'WA Fleitas Amnesty Oath', 'Walhalla', 'Woodville Wilkinson Co'],
      dtype='<U41')

****

In [50]:
np.unique(LOC)

array(['South Ca'], dtype='<U8')

In [51]:
np.unique(LOC1)

array(['AMERICA', 'So Carolina', 'South Carolina', 'US', 'United States'],
      dtype='<U14')

In [52]:
np.unique(LOC2)

array(['Aiken', 'Aiken Burnwell', 'Charleston', 'Charleston Sate',
       'Charleston Sc', 'City', 'City of Charleston',
       'City of New Orleans', 'County', 'County of Charleston',
       'County of Orangeburg', 'District of Beaufort', 'Gainsville',
       'Harrison Co', 'Marshal City of Charleston', 'Miss', 'Miss City',
       'Mississippi City', 'NC', 'Oacen Springs', 'Orangeburg SC',
       'Pass Christian Mississippi', 'Port of Miss', 'SC', 'South Ca',
       'State of So Carolina', 'State of South Carolina',
       'UNITED STATES OF AMERICA', 'United State of America',
       'United States', 'United States Union of State',
       'United States Union of States', 'United States of America',
       'Walhalla State of South Carolina', 'Warren Co',
       'Wilkinson County State of Mississippi'], dtype='<U37')

****

In [53]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '48 year', '58 year', '58 year of age',
       '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 20 1865', 'Aug 20th 1865',
       'Aug 31 1865', 'August 24 1865', 'August 29th 1865', 'July 22',
       'Maj 47', 'Maj 47th', 'Maj 47th PM', 'May 29th 1865', 'May 4',
       'May 47 PM', 'Sept 29 1865', 'Sept 29th 1865', 'Thirty nine',
       'twenty eight year'], dtype='<U26')

In [54]:
np.unique(GP)

array(['Charleston', 'Charleston S C', 'Charleston SC',
       'Mississippi City', 'Orangeburg', 'So Carolina', 'South Carolina',
       'UNITED STATES OF', 'US', 'United State of America',
       'United States', 'United States of America'], dtype='<U24')

# Post processing

**For dates**

In [55]:
import regex as re
Pattern1=re.compile(r'^\d\.*.*\d\d$')
Pattern2=re.compile(r'^[A-Z]\.*.*\d \.*.*\d\d$')

In [56]:
M=[]
for i in Date:
        date=re.search(r'^\d\.*.*\d\d$' ,i)
        date2=re.search(r'^[A-Z]\.*.*\d \.*.*\d\d$' ,i)
        M.append(date)
        M.append(date2)
       

In [57]:
Final_date=[]
for i in M:
    if i!=None:
        Final_date.append(i.group())

In [58]:
Final_date


['27th day of Novb 1865',
 '27th day of Novbr 1865',
 '28 day of Sept 1865',
 '31 day of Oct 1865',
 '31st day of August AD 1865',
 'August 24 1865',
 '3 day of August 1865',
 '3 day of Augt 1865',
 '16th day of Sept 1865',
 '16 day of October 1865',
 '6 day of Nov 1865',
 'Sept 29 1865',
 '29th day of September 1865',
 '29th day of Sept 29 1865',
 '3d day of February 1868',
 '7th day of September 186',
 'Aug 20 1865',
 '20th day of August 1865',
 '22nd day of August 1865',
 '18th Sept 1865',
 'Aug 31 1865',
 '31st day of Aug 1865',
 '31 day of Aug 1865']

**great !**

# best result obtained 

***Person***

In [59]:
np.unique(P2)

array(['ANDREW JOHNSON', 'Almighty', 'Almighty God', 'Andrew Johnson',
       'Anne H Elliott', 'Caroline Belitzer', 'E C Whaley', 'EB Towne',
       'Frank Conner', 'Frank E Miller', 'Frank Geise', 'GOD',
       'Geo D Carpenter', 'God', 'H Elliott', 'H F Holwell',
       'Hiram Smith', 'Hu Kennedy', 'I E C Whaley', 'I J F Steinmeyer',
       'J E Thuring', 'J H Mathews', 'JB Hiserodt', 'JF Steinmeyer',
       'JH Mathews', 'JW Baxter', 'Jacob Schroder', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Joseph Edings Sworn',
       'Levi Steuben', 'Levi Stuber', 'Mrs E Montgomery', 'P H Joor',
       'P P Bergevin', 'R A Bray', 'Robert A Wilson', 'S Wilde Durson',
       'S Wilde Durson Magt', 'S Wilde Durson Magtr', 'Sgd Montgomery',
       'William Whaley', 'Wm Whaley', 'caroline Belitzer'], dtype='<U20')

****

***Organization***

In [60]:
np.unique(OR2)


array(['128th USCT', '47th Pa', 'AMNESTY OATH', 'Amnesty', 'Amnesty Oath',
       'Amnesty Proclamation', 'Constitution', 'Emancipation of Slaves',
       'Emancipation of SlavesSO', 'Emancipation of SlavesSo',
       'MARSHALS OFFICE', 'NYVV', 'Oath',
       'Thomas Andrew Oath of Loyalty Recd Bureau',
       'Thos Gibbs Henderson Co', 'U S C', 'UNITED STATES FORCES', 'US',
       'US Cold Inf', 'US Cold Infty', 'USCI', 'USCT', 'Union', 'Vols',
       'WA Fleitas Amnesty Oath', 'Walhalla', 'Woodville Wilkinson Co'],
      dtype='<U41')

****

***GPE and location***

In [61]:
np.unique(GPE)

array(['Charleston', 'Gainsville', 'Mississippi', 'Mississippi City',
       'NC', 'New Orleans', 'South Carolina', 'States', 'US',
       'United States', 'United States of America', 'Vicksburg'],
      dtype='<U24')

In [62]:
np.unique(LOC2)

array(['Aiken', 'Aiken Burnwell', 'Charleston', 'Charleston Sate',
       'Charleston Sc', 'City', 'City of Charleston',
       'City of New Orleans', 'County', 'County of Charleston',
       'County of Orangeburg', 'District of Beaufort', 'Gainsville',
       'Harrison Co', 'Marshal City of Charleston', 'Miss', 'Miss City',
       'Mississippi City', 'NC', 'Oacen Springs', 'Orangeburg SC',
       'Pass Christian Mississippi', 'Port of Miss', 'SC', 'South Ca',
       'State of So Carolina', 'State of South Carolina',
       'UNITED STATES OF AMERICA', 'United State of America',
       'United States', 'United States Union of State',
       'United States Union of States', 'United States of America',
       'Walhalla State of South Carolina', 'Warren Co',
       'Wilkinson County State of Mississippi'], dtype='<U37')

****

**Date**

In [63]:
Final_date

['27th day of Novb 1865',
 '27th day of Novbr 1865',
 '28 day of Sept 1865',
 '31 day of Oct 1865',
 '31st day of August AD 1865',
 'August 24 1865',
 '3 day of August 1865',
 '3 day of Augt 1865',
 '16th day of Sept 1865',
 '16 day of October 1865',
 '6 day of Nov 1865',
 'Sept 29 1865',
 '29th day of September 1865',
 '29th day of Sept 29 1865',
 '3d day of February 1868',
 '7th day of September 186',
 'Aug 20 1865',
 '20th day of August 1865',
 '22nd day of August 1865',
 '18th Sept 1865',
 'Aug 31 1865',
 '31st day of Aug 1865',
 '31 day of Aug 1865']

****